# Rosalind Bioinformatics Stronghold

The website http://rosalind.info has a number of valuable programming exercises to help teach core bioinformatics concepts. To give you practice with writing and running python scripts, a command line version of the Rosalind Stronghold problems has been created for you. Some labs will have a Rosalind problem for you to solve as part of the lab, and this is the first such lab.

## Installing rosalind
First let's install rosalind in an anaconda environment on SCC from <a hre="https://anaconda.org/bubhub/rosalind-stronghold">BUBHUB</a>. 

`module load anaconda3`

`conda create -n rosalind python=3`

`source activate rosalind`

`conda install -c bubhub rosalind-stronghold`


## Running rosalind

To get started, run the following on the command line:

`rosalind help`

The information printed to the screen will give you instructions on how to use the `rosalind` program.

The rosalind problem ID you will solve today is **DNA**, where you are asked to print out the counts of each nucleotide in a specific format. Run:

`rosalind info DNA`

to read a description of the problem and see an example input and expected output. To save the python template at the end of the info, run the following:

`rosalind info DNA > rosalind_DNA_template.py`

You can then edit the script `rosalind_DNA_template.py` to complete the task. When you think your script is ready to test, run:

`rosalind submit DNA rosalind_DNA_template.py`

The program will show you how your script output compares with the expected output.

Submit your python script to Blackboard for credit.

# Using Bowtie 2

Today we will learn how to use the Bowtie 2 aligner. As we discussed in class, aligners use indices to efficiently find the region where a read aligns.  We will use Bowtie 2 to build an index, and then align reads to that index.  You will learn how to build indices on genomes and how to align reads to those genomes.  You will also learn how to view the alignment results using samtools. This lab is based on the tutorial for bowtie2, on the bowtie website.

**For this lab you will need to be logged into SCC!** You will execute commands in your shell as they are written in this document. Remember, you must run the commands that are shown in the order they are shown or they will fail. 

To list all available bowtie versions on SCC:

`module avail bowtie`

To load the Bowtie2 module on SCC: 
module load 

`module load bowtie2`


## Part A: Indices

Last lab, we learned about dictionaries in Python. An index is similar to a dictionary, since we can use it to look up the location of a query sequence in a genome. Indices are used by Bowtie for memory compression and to speed up searching through the genome.  Let’s create and initialize an index on the genome of a lambda phage.

1. Make a directory for lab10 in your own folder on `/projectnb/bf527/`.

1. Copy the content of this lab from `/project/bf527/jupyter/BF527_lab_10_content/` to your lab folder. You will need the `lambda_phage.zip` compressed files. 

   `cp /project/bf527/jupyter/BF527_lab_10_content/lambda_phage.zip ./`

1. This is a zip archive, which you can unzip with:

  `unzip lambda_phage.zip`
   
1. There should now be a new directory in the directory you are in named `lambda_phage_example`. Use cd to enter this directory.
1. In this new directory, you should see the following directories after running `ls`:

   `index  reads  reference`
   
1. We will now build the index in the `index` directory using the following command:

   `bowtie2-build reference/lambda_virus.fa index/lambda_virus`
   
   What this command is doing is using the program `bowtie2-build` to build and index named `lambda_virus` from the fasta file `reference/lambda_virus.fa` into the `index` directory. 
   
1. Type `ls index`. You should see a list of files that start with the name `lambda_virus` and have various endings. These files make up your new index! To check to make sure our index was created properly, run the `bowtie2-inspect` program to print out the names of the sequences in the index:

   `bowtie2-inspect -n index/lambda_virus`
   
   **Copy the output of the `bowtie2-inspect` command in the box below**

## Part B: Aligning reads

Now that we have an index, we can use the index to align reads against our genome. We will stay in the `lambda_phage_example` directory for the following steps.

1. Reads generated by high-throughput sequencing technologies are often provided in [FASTQ format](https://en.wikipedia.org/wiki/FASTQ_format). There are example FASTQ formatted files with short reads from the lambda phage genome in the archive you uploaded to enggrid. Briefly look at the format of one of these files using the following command:

   `head reads/reads_1.fq`
   
   You don't need to understand all the contents of this file for this lab, but if you are curious the [FASTQ Wikipedia article](https://en.wikipedia.org/wiki/FASTQ_format) is a good reference.
   
1. We are now going to align the reads from this file against the reference we just built. Run this command:

   `bowtie2 -x index/lambda_virus -U reads/reads_1.fq -S lp1.sam`
   
   You should see some output from running this command. The `-x` option tells bowtie the path to the index to use. `-U` tells it to use unpaired reads, and gives the name of the file the reads are in, and `-S lp1.sam` tells it to write the alignments to the file `lp1.sam` in [SAM format](https://samtools.github.io/hts-specs/SAMv1.pdf).
   
1. Run the command `head lp1.sam` to see the beginning of the file. This file is in SAM format - we will learn how to interpret SAM files later in the lab.

1. There are two types of sequencing reads - single-end and paired-end. In the sequencing sample prep protocol, the DNA to be sequenced is sheared into short fragments, usually of 300-500nt. Single-end sequencing produces the sequence of only one end these fragments, while paired-end sequencing produces the sequence from both ends of the fragments. The sequences from paired-end datasets are stored in two different files that have the same number of reads in each, where the first read in each file form a mated pair of sequences originating from the same fragment, the second read in each file form a mated pair for another fragment, and so on.
   
   The file `reads_1.fq` from above is actually one of the files from a paired-end sequencing experiment, and an additional file `reads_2.fq` is also in the examples directory. Use the following to read the usage information for `bowtie2`:

   `bowtie2 -h | less`

    **Construct the correct command line command to run bowtie2 in paired-end mode using the files `reads_1.fq` and `reads_2.fq`, and output the alignments to `lp2.sam`.**

## Part C: Analyzing SAM files

Now that we’ve generated some SAM files, we need to be able to view and interpret our results. From the format description:

> SAM stands for Sequence Alignment/Map format. It is a TAB-delimited text format consisting of a header section, which is optional, and an alignment section. If present, the header must be prior to the alignments. Header lines start with `@`, while alignment lines do not. Each alignment line has 11 mandatory fields for essential alignment information such as mapping position, and variable number of optional fields for flexible or aligner specific information.
    
We will be using the samtools toolset to process the SAM files generated by Bowtie.

1. The first thing we will do is convert the SAM file into a BAM file. A BAM file is equivalent to a SAM file, but uses a more efficient indexed and compressed binary format.

   `samtools view -bS lp2.sam > lp2.bam`
 
1. We will turn this file into a sorted BAM file. Sorted BAM files make it easy to perform a variant analysis, which is what we will be doing next:

   `samtools sort lp2.bam -o lp2.sorted.bam`
 
1. Now we will use samtools to generate a VCF file of the variants that were found. A VCF file is the standard file format used to describe variants found during sequence analysis. Make sure that you type the following command all on one line.

   `samtools mpileup -uf reference/lambda_virus.fa lp2.sorted.bam | bcftools call -O b -vc --ploidy 1 > lp2.raw.bcf`
 
1. Finally, we will use bcftools to view the VCF file:

   `bcftools view lp2.raw.bcf`
   
**Copy the first three lines of the output into the following box.** You don't need to interpret the output of this VCF file.

## Lab Task

 Use `bowtie2-build` to create an index for a genome, and then `bowtie2` to align reads to that genome. Then, using `samtools`, view the alignments and pass to `bcftool` to call variants. Use this information to determine if there are indels in the reads that you have aligned. 
 
### Objectives:

1.  If you followed the instructions printed to the screen after running `source activate Lab_10_Bowtie`, you should have the files `ecolicompletegenome.fasta` and `query_seqs.fasta` (along with others) in your lab directory.

2. Create a new folder named `ecoli`, and in it make an index for the E. Coli genome using `ecolicompletegenome.fasta` and `bowtie2-build`.

3. Run the `bowtie2` aligner, using the query sequences in `query_seqs.fasta`, against the index you created in step 2. Hint: Use `-f` to use fasta instead of fastq

4. Use `samtools` to generate a BAM file and a sorted BAM file, and use `bcftools call` to call variants. Then use `bcftools view` as above to look at the results.

5. What did you find? Paste the output alignment as well as a brief statement into the box below. 